In [9]:
import pandas as pd
import re
import numpy as np

In [10]:
df_shopping = pd.read_csv('../data/raw/shopping_center.csv',header = None)
df_shopping.head()
df_shopping = df_shopping.loc[:,[0,1]]
df_shopping.head()
df_shopping = df_shopping.values
Y = []
X = []
for info in df_shopping[:,1]:
  Y.append(re.findall(r"\d+\.?\d*",df_shopping[0,1])[0])
  X.append(re.findall(r"\d+\.?\d*",df_shopping[0,1])[1])
Y = np.asarray(Y,dtype = float).reshape(-1,1)*-1.0
X = np.asarray(X,dtype = float).reshape(-1,1)
df_shopping = np.concatenate((df_shopping[:,0].reshape(-1,1),X,Y),axis = 1)
df_shopping = pd.DataFrame(df_shopping)
df_shopping.columns = ['Shopping Name','Longitude','Latitude']
df_shopping.head()

,Shopping Name,Longitude,Latitude
0,206 BOURKE STREET,144.967,-37.8127
1,ACLAND COURT SHOPPING CENTRE,144.967,-37.8127
2,ALTONA GATE SHOPPING CENTRE,144.967,-37.8127
3,BALNARRING VILLAGE SHOPPING CENTRE,144.967,-37.8127
4,BARKLY SQUARE,144.967,-37.8127


In [14]:
centroid = pd.read_csv('../data/curated/centroid.csv')
centroid = centroid.drop(columns='Unnamed: 0')
centroid = centroid.dropna()
centroid.head()

,Postcode,Centroid
0,3000,-37.817403 144.956776
1,3001,-37.817403 144.956776
2,3002,-37.818517 144.982207
3,3003,-37.810871 144.949592
4,3004,-37.844246 144.970161
